# Project Notebook: Augmenting Pandas with SQLite

## Question 1: Introduction

In this session, we explored a few different ways to work with larger datasets in pandas. In this guided project, we'll practice using some of the techniques we learned to analyze startup investments from Crunchbase.com.

Every year, thousands of startup companies raise financing from investors. Each time a startup raises money, we refer to the event as a fundraising round. Crunchbase is a website that crowdsources information on the fundraising rounds of many startups. The Crunchbase user community submits, edits, and maintains most of the information in Crunchbase.

In return, Crunchbase makes the data available through a Web application and a fee-based API. Before Crunchbase switched to the paid API model, multiple groups crawled the site and released the data online. Because the information on the startups and their fundraising rounds is always changing, the data set we'll be using isn't completely up to date.

Throughout this project, we'll practice working with different memory constraints. In this step, let's assume we only have 10 megabytes of available memory. While crunchbase-investments.csv (https://bit.ly/3BPcobU) consumes 10.3 megabytes of disk space, we know from earlier lessons that pandas often requires 4 to 6 times amount of space in memory as the file does on disk (especially when there's many string columns).


**Tasks**

* Because the data set contains over 50,000 rows, you'll need to read the data set into dataframes using 5,000 row chunks to ensure that each chunk consumes much less than 10 megabytes of memory.
* Across all of the chunks, become familiar with:
1. Each column's missing value counts.
2. Each column's memory footprint.
3. The total memory footprint of all of the chunks combined.
4. Which column(s) we can drop because they aren't useful for analysis.



In [1]:
# Your code goes here
# Dataset URL = https://bit.ly/3BPcobU
#

import pandas as pd

df = pd.read_csv('https://bit.ly/3BPcobU', encoding = "ISO-8859-1", chunksize=5000)


In [4]:
# Check columns for missing values

df = pd.read_csv('https://bit.ly/3BPcobU', encoding = "ISO-8859-1", chunksize=5000)
for chunk in df:
  print(chunk.isnull().sum())  

company_permalink            0
company_name                 0
company_category_code       52
company_country_code         0
company_state_code          53
company_region               0
company_city                64
investor_permalink           0
investor_name                0
investor_category_code    2557
investor_country_code      778
investor_state_code       1371
investor_region              0
investor_city              900
funding_round_type           0
funded_at                    0
funded_month                 0
funded_quarter               0
funded_year                  0
raised_amount_usd          653
dtype: int64
company_permalink            0
company_name                 0
company_category_code       51
company_country_code         0
company_state_code          43
company_region               0
company_city                45
investor_permalink           0
investor_name                0
investor_category_code    5000
investor_country_code      261
investor_state_code       

In [15]:
# Check each columns memory footprint

chunk_iter = pd.read_csv('https://bit.ly/3BPcobU', encoding = "ISO-8859-1", chunksize=5000)
for chunk in chunk_iter:
  for col in chunk:
    print(chunk.memory_usage(deep=True)/(1024*1024))

Index                     0.000122
company_permalink         0.365729
company_name              0.323832
company_category_code     0.308253
company_country_code      0.286102
company_state_code        0.279969
company_region            0.308153
company_city              0.315244
investor_permalink        0.406121
investor_name             0.340663
investor_category_code    0.228371
investor_country_code     0.265327
investor_state_code       0.246032
investor_region           0.306885
investor_city             0.286545
funding_round_type        0.305766
funded_at                 0.319481
funded_month              0.305176
funded_quarter            0.305176
funded_year               0.038147
raised_amount_usd         0.038147
dtype: float64
Index                     0.000122
company_permalink         0.365729
company_name              0.323832
company_category_code     0.308253
company_country_code      0.286102
company_state_code        0.279969
company_region            0.308153
compa

In [18]:
# Calculate the total memory usage across all of the chunks.

memory_footprints = 0 
chunk_iter = pd.read_csv('https://bit.ly/3BPcobU', encoding = "ISO-8859-1", chunksize=5000)
for chunk in chunk_iter:
  memory_footprints += (chunk.memory_usage(deep=True).sum()/(1024*1024))
print(f"{memory_footprints} MB")

56.988911628723145 MB


We can drop 'investor_category_code' column as it is missing most values hence not useful for analysis.

## Question 2: Selecting Data Types

Now that we have a good sense of the missing values, let's get familiar with the column types before adding the data into SQLite.

**Tasks**

* Identify the types for each column.
* Identify the numeric columns we can represent using more space efficient types.
For text columns:
* Analyze the unique value counts across all of the chunks to see if we can convert them to a numeric type.
* See if we clean clean any text columns and separate them into multiple numeric columns without adding any overhead when querying.
* Make your changes to the code from the last step so that the overall memory the data consumes stays under 10 megabytes.


In [20]:
# Your code goes here

chunk_iter = pd.read_csv('https://bit.ly/3BPcobU', encoding = "ISO-8859-1", chunksize=5000)
for chunk in chunk_iter:
  chunk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   company_permalink       5000 non-null   object 
 1   company_name            5000 non-null   object 
 2   company_category_code   4948 non-null   object 
 3   company_country_code    5000 non-null   object 
 4   company_state_code      4947 non-null   object 
 5   company_region          5000 non-null   object 
 6   company_city            4936 non-null   object 
 7   investor_permalink      5000 non-null   object 
 8   investor_name           5000 non-null   object 
 9   investor_category_code  2443 non-null   object 
 10  investor_country_code   4222 non-null   object 
 11  investor_state_code     3629 non-null   object 
 12  investor_region         5000 non-null   object 
 13  investor_city           4100 non-null   object 
 14  funding_round_type      5000 non-null   

## Question 3: Loading Chunks Into SQLite

Now we're in good shape to start exploring and analyzing the data. The next step is to load each chunk into a table in a SQLite database so we can query the full data set.

**Tasks**

1. Create and connect to a new SQLite database file.
2. Expand on the existing chunk processing code to export each chunk to a new table in the SQLite database.
3. Query the table and make sure the data types match up to what you had in mind for each column.

In [2]:
# Your code goes here

import sqlite3
import pandas as pd
conn = sqlite3.connect('crunchbase.db')
invest_iter = pd.read_csv('https://bit.ly/3BPcobU', encoding = "ISO-8859-1", chunksize=5000)
for chunk in invest_iter:
    chunk.to_sql("investments", conn, if_exists='append', index=False)

result = pd.read_sql('PRAGMA table_info(investments);', conn)
print(result)

    cid                    name     type  notnull dflt_value  pk
0     0       company_permalink     TEXT        0       None   0
1     1            company_name     TEXT        0       None   0
2     2   company_category_code     TEXT        0       None   0
3     3    company_country_code     TEXT        0       None   0
4     4      company_state_code     TEXT        0       None   0
5     5          company_region     TEXT        0       None   0
6     6            company_city     TEXT        0       None   0
7     7      investor_permalink     TEXT        0       None   0
8     8           investor_name     TEXT        0       None   0
9     9  investor_category_code     TEXT        0       None   0
10   10   investor_country_code     TEXT        0       None   0
11   11     investor_state_code     TEXT        0       None   0
12   12         investor_region     TEXT        0       None   0
13   13           investor_city     TEXT        0       None   0
14   14      funding_roun

## Question 4: Next Steps

Now that the data is in SQLite, we can use the pandas SQLite workflow we learned in the last lesson to explore and analyze startup investments. Remember that each row isn't a unique company, but a unique investment from a single investor. This means that many startups will span multiple rows.

Use the pandas SQLite workflow to answer the following questions:

* What proportion of the total amount of funds did the top 10% raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised.
* Which category of company attracted the most investments?
* Which investor contributed the most money (across all startups)?
* Which investors contributed the most money per startup?
* Which funding round was the most popular? Which was the least popular?

Here are some ideas for further exploration:

* Repeat the tasks in this project using stricter memory constraints (under 1 megabyte).
* Clean and analyze the other Crunchbase data sets from the same GitHub repo.
* Understand which columns the data sets share, and how the data sets are linked.
* Create a relational database design that links the data sets together and reduces the overall disk space the database file consumes.

Use pandas to populate each table in the database, create the appropriate indexes, and so on.

In [28]:
# Your code goes here
q = 'select investor_name, sum(raised_amount_usd) as Raised_funds from investments group by investor_name order by sum(raised_amount_usd) desc;'
Raised = pd.read_sql(q, conn)
top_ten = len(Raised['investor_name'])/10
print(Raised[:20])

                       investor_name  Raised_funds
0   Kleiner Perkins Caufield & Byers  2.243565e+10
1          New Enterprise Associates  1.938508e+10
2                     Accel Partners  1.294425e+10
3                      Goldman Sachs  1.275092e+10
4                    Sequoia Capital  1.207880e+10
5                              Intel  1.193840e+10
6                             Google  1.161760e+10
7                        Time Warner  1.146000e+10
8                            Comcast  1.133800e+10
9                  Greylock Partners  9.921966e+09
10                       BrightHouse  9.400000e+09
11                     Intel Capital  9.391234e+09
12     Draper Fisher Jurvetson (DFJ)  9.002922e+09
13           Oak Investment Partners  8.700129e+09
14               Andreessen Horowitz  8.467348e+09
15       Lightspeed Venture Partners  8.281958e+09
16                         Benchmark  8.239898e+09
17                  Battery Ventures  8.109063e+09
18                 Redpoint Ven

In [16]:
# Which category of company attracted the most investments?

q = 'select company_category_code, sum(raised_amount_usd) as Raised_funds from investments group by company_category_code order by sum(raised_amount_usd) desc;'
Raised = pd.read_sql(q, conn)
print(Raised[:10])

  company_category_code  Raised_funds
0               biotech  2.207928e+11
1              software  1.461690e+11
2                mobile  1.295548e+11
3             cleantech  1.054105e+11
4            enterprise  9.172185e+10
5                   web  8.028653e+10
6               medical  5.073421e+10
7           advertising  5.015332e+10
8             ecommerce  4.513444e+10
9       network_hosting  4.483937e+10


Biotech attracted the most investments

In [18]:
# Which investor contributed the most money (across all startups)?

q = 'select investor_name, sum(raised_amount_usd) as Raised_funds from investments group by investor_name order by sum(raised_amount_usd) desc;'
Raised = pd.read_sql(q, conn)
print(Raised[:5])

                      investor_name  Raised_funds
0  Kleiner Perkins Caufield & Byers  2.243565e+10
1         New Enterprise Associates  1.938508e+10
2                    Accel Partners  1.294425e+10
3                     Goldman Sachs  1.275092e+10
4                   Sequoia Capital  1.207880e+10


Kleiner Perkins Caufield & Byers contributed the most money across all startups.

In [27]:
# Which investors contributed the most money per startup?

q = 'select company_name, investor_name, sum(raised_amount_usd) as Raised_funds from investments group by investor_name, company_name order by sum(raised_amount_usd) desc;'
Raised = pd.read_sql(q, conn)
print(Raised[:20])

      company_name                     investor_name  Raised_funds
0        Clearwire                           Comcast  1.124000e+10
1        Clearwire                             Intel  1.124000e+10
2        Clearwire                       Time Warner  1.124000e+10
3        Clearwire                       BrightHouse  9.400000e+09
4        Clearwire                            Google  6.400000e+09
5        sigmacare            Marlin Equity Partners  5.200000e+09
6        Clearwire                     Sprint Nextel  5.000000e+09
7        Clearwire              Eagle River Holdings  4.840000e+09
8         Facebook          Digital Sky Technologies  3.400000e+09
9         Facebook                     Goldman Sachs  3.000000e+09
10         Groupon                  Battery Ventures  2.170000e+09
11         Groupon          Digital Sky Technologies  2.170000e+09
12  Wave Broadband                       GI Partners  2.100000e+09
13  Wave Broadband         Oak Hill Capital Partners  2.100000

Comcast, Intel, Time Warner, BrightHouse and Google contributed the most money per startup.

In [24]:
# Which funding round was the most popular? Which was the least popular?

q = 'select funding_round_type, sum(raised_amount_usd) as Raised_funds from investments group by funding_round_type order by sum(raised_amount_usd) desc;'
Raised = pd.read_sql(q, conn)
print(Raised[:10])


  funding_round_type  Raised_funds
0          series-c+  5.315069e+11
1            venture  2.611130e+11
2           series-b  2.566536e+11
3           series-a  1.730843e+11
4           post-ipo  6.183520e+10
5              other  3.701452e+10
6     private-equity  3.231975e+10
7              angel  9.924150e+09
8       crowdfunding  1.298300e+07
9               None           NaN


series-c+ was the most popular funding round. Crowdfunding was the least popular.